In [127]:
import openai
import os
import re

In [128]:
os.chdir('C:/Users/kaosh/Desktop/Side-project-ML/sentence_analyze')  # Change to your own path

In [129]:
import openpyxl
wb = openpyxl.load_workbook('TCB Response_criteria.xlsx', data_only=True)     # Open Excel file



In [130]:
s1 = wb['DevOps1']        # Get the worksheet content

In [131]:
# Store all Capabilities in an array
s1_capability_read = s1.iter_rows(max_col=1)
capability_read = []
for row in s1_capability_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    capability_read.append(tmp)

# Store all questions in an array
s1_question_read = s1.iter_rows(min_col=2, max_col=2)
question_read = []
for row in s1_question_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    question_read.append(tmp)

# Store all answer 
s1_answer_read = s1.iter_rows(min_col=3, max_col=3)
answer_read = []
for row in s1_answer_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    answer_read.append(tmp)
    
# Store all criteria in an array
s1_criteria_read = s1.iter_rows(min_col=4, max_col=4)
criteria_read = []
for row in s1_criteria_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    criteria_read.append(tmp)


In [132]:
# Set your OpenAI API key
openai.api_key = '...'

In [133]:
# The analyze_sentence function sends a request to the OpenAI API with the given sentence and criteria, and returns the decision made by the model.
def analyze_sentence(sentence, criteria):
    response = openai.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Sentence: {sentence}\nCriteria: {criteria}\nDecision:",
        temperature=0.7,
        max_tokens=100,
        stop=["\n"]
    )
    decision = response.choices[0].text.strip()
    return decision

In [134]:
def summarize_paragraph(paragraph):
    # Use the completion endpoint to summarize the paragraph
    response = openai.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f"Write a concise summary of the following: {paragraph}",
      max_tokens=100,
      temperature=0.5,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    # Extract and return the summarized text from the response
    return response.choices[0].text.strip()

In [135]:
# Find out all keywords of the criteria
def criteria_keywords(criteria):
    response = openai.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f"Find keyword or key information of the following: {criteria}",
      max_tokens=100,
      temperature=0.5,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    return response.choices[0].text.strip()

In [136]:
def Average(lst): 
    return sum(lst) / len(lst)

In [137]:
keywords_array = []
for i in range(0, len(criteria_read), 2):
    keywords = criteria_keywords(criteria_read[i])
    print(keywords)
    keywords_array.append(keywords)
    keywords_array.append(keywords)

1. Standardized toolset
2. Comprehensive toolset
3. Organization-wide standards
4. Unified processes
5. Integrated tooling
6. Support for application development
7. Deployment tool standardization
8. Infrastructure templates
9. Standardized templates
10. Consistent deployment
11. Infrastructure efficiency
12. Mature approach
13. Tooling decisions
14. Infrastructure needs
15. Standardized processes
16. Development and deployment
17. Unified infrastructure
Shared Repository, CI/CD Practices, Centralization, Automation, Efficiency, Reliability, Automated Builds, Testing, Consistency, Management, Uniformity, Organization, Relevant Teams, Scripting, Deployment
Keyword/key information: Shared repository, CI/CD practices, centralization, automation, efficiency, reliability, automated builds, testing, consistency, management, uniformity, organization.
1. Automated Compliance Impact
2. Improvement in compliance posture
3. Scripted Remediations
4. Effectiveness of scripted compliance actions
5. 

In [138]:
# The main function prompts the user to input the sentence and criteria they want to analyze, calls analyze_sentence with these inputs, and prints out the decision.
run_times = 10 # 期望跑的次數
result = [[] for _ in range(len(answer_read)+1)]
result[0].append("Capability")
result[0].append("Questions")
result[0].append("All Answers concatenate")
result[0].append("Criteria")
result[0].append("Keywords of criteria")
for run_time in range(run_times):
    result[0].append(str(run_time + 1))
result[0].append("Average Score")
result[0].append("Final Justification")
for i in range(len(answer_read)):
    result[i+1].append(capability_read[i][0]) # Put capability array in result
    result[i+1].append(question_read[i][0]) # Put Questions array in result
    result[i+1].append(answer_read[i][0])   # Put all answer array in result
    result[i+1].append(criteria_read[i][0])   # Put all criteria array in result 
    result[i+1].append(keywords_array[i])   # Put all keywords array in result 
    
    All_justification = "" # Store all justification made by GPT
    for run_time in range(run_times):
        if answer_read[i][0] == None:
            result[i+1].append("")
        else: 
            if answer_read[i][0].strip("/") == "":
                result[i+1].append("")
            else:
                sentence = answer_read[i][0]
                criteria = "Can you describe how the sentences meet the criteria as you are a consultant?: " + keywords_array[i]
                decision = analyze_sentence(sentence, criteria)
                result[i+1].append(f'{decision}')
                All_justification = All_justification + f'{decision}' + "/" # Concatenance all justification into a paragraph 
                #print(f'{decision}')
    temp = re.findall(r'\d+%', All_justification)
    res = []
    for n in temp:
        res.append(int(n.strip('%')))
    if len(res) == 0:
        result[i+1].append("")
    else:
        result[i+1].append(Average(res))
        print(Average(res))
    if All_justification == "":
        result[i+1].append("")
    else:
        summary= summarize_paragraph(All_justification)
        result[i+1].append(summary)
        print(summary)

These sentences meet the criteria as they describe the use of a standardized toolset, DMS, for querying and checking document completeness in the deployment process. This toolset is comprehensive and supports both development and deployment needs, while also promoting standardized processes and decision-making. Additionally, the mention of TFS for automatic deployment and the use of infrastructure templates demonstrate a mature approach to tooling decisions and infrastructure needs. Overall, the sentences illustrate a unified and efficient approach to deployment, meeting organization-wide standards and ensuring consistency
These sentences demonstrate an understanding of the importance of implementing processes and tools for efficiency and reliability in software development. The use of a shared repository promotes centralization and consistency in code, while CI/CD practices and automation increase efficiency and reliability. Automated builds and testing ensure consistency and reliabil

In [139]:
# Save the results in excel '...'
excel = openpyxl.Workbook('Innovation_Result.xlsx')     
s3 = excel.create_sheet() 
for w in result:
    s3.append(w)
excel.save('Innovation_Result.xlsx')